<a href="https://colab.research.google.com/github/Renshui-MC/DeepLearning-ZerosToGans/blob/main/PyTorch%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Linear regression using PyTorch built-ins**

* Gradient descent can be done very eaily using `loss.backward()`. Otherwise, we must implement it using `Numpy`
* `Numpy` does not support GPUs but `PyTorch` does (when we deal with large datasets we need to move them to GPUs)
* Instead of implementing a regression model from the scratch using `Numpy`, `PyTorch` provides basic building blocks that can easily build a regression model that requires only a few lines of code
* **linear regression** is the simplest form of neural network

In [4]:
import torch.nn as nn
import numpy as np
import torch

Again first things first we need training dataset: input features and ouput targets. 

+ from outside in the number of square brackets means how many dimensions you have. One square bracket means 1D, and two square brackets mean 2D.

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)#convert to torch data type
targets = torch.from_numpy(targets)#convert to torch data type

We are using larger datasets of 15 training examples

In [6]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

##**Dataset and DataLoader**

In practice, we might have millions of data, and it is difficult to print all data at once due to too slow to print. Instead, we want to take the data down into **batches**. That helps us train our models faster. 

+ create a `TensorDataset` to access rows from inputs and targets
+ `TensorDataset` also provides **standard APIs** that can work with many different types of datasets in pyTorch
+ `TensorDataset` can access **a small section** of the training dataset using the array indexing notation, e.g., `[0:3]` (first three rows or training examples). You will obtain **two outputted elements**. For `[0:3]`, you will get first three rows for inputs (first element) and first three rows for targets (second element).
+ `TensorDataset` is a class in `PyTorch` 


In [7]:
from torch.utils.data import TensorDataset

In [38]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3] #just want to take a look at the first three rows of data. But remember we actually have 15 rows

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

Then we use `DataLoader` to split the data we just defined using `TensorDataset` into **batches**.

+ set batch size to $5$ (**number of rows or training examples**) then you have **total training examples/batch training examples $= 15/5=3$ batches**
+ **a batch** contains rows of data based on the specified batch size. You can have **a batch of inputs** and **a batch of outputs**
+ set `shuffle = True` to create random data before creating batches, which is more efficient for training, e.g., faster reduction in the loss. If you compare the shuffled data with original inputs or outputs in the same specified rows, you will notice that the data are different from each other
+ loop through `train_dl` (`DataLoader`) and return batches of data with the given batch size (`TensorDataset`)
+ `break` ensures only the first batch is printed


In [9]:
from torch.utils.data import DataLoader

In [33]:
# Define data loader
batch_size = 5 
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
#print(train_dl.dataset[1])

In [39]:
#iterate through the 3 batches (batch size  = 5)
for xb, yb in train_dl: 
  print(xb)
  print(yb)
  break # we only need the first batch

tensor([[ 69.,  96.,  70.],
        [ 88., 134.,  59.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [ 68.,  97.,  70.]])
tensor([[103., 119.],
        [118., 132.],
        [ 82., 100.],
        [118., 134.],
        [102., 120.]])


In [21]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

##**nn.Linear**
Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from `PyTorch`, which does it automatically. **Here we are using a linear layer of NN.**

+ define the model `nn.Linear(3,2)`, where $3$ represents the number of features and $2$ represents the number of targets (we here have two targets: Apple and Orange)
+ two targets imply that we have two sets of weights (one set has $3$ weights that is equal to the number of features, and therefore, we have $3 \times 2 = 6$ weights in total) **It is just the number of features multiplied by the number of targets**
+ two targets need $2$ biases
+ `nn.Linear` automatically outputs weights and biases for us

In [22]:
# create a linear regression model using PyTorch
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[0.2578, 0.5727, 0.2086],
        [0.3338, 0.4693, 0.5319]], requires_grad=True)
Parameter containing:
tensor([-0.5341, -0.3569], requires_grad=True)


##**Parameters**
`PyTorch` models have a usefull method called `.parameters()`. It presents all weights and biases in matrices.

In [40]:
list(model.parameters())

[Parameter containing:
 tensor([[0.2578, 0.5727, 0.2086],
         [0.3338, 0.4693, 0.5319]], requires_grad=True), Parameter containing:
 tensor([-0.5341, -0.3569], requires_grad=True)]

Now we can generate predictions using our `PyTorch` model

In [41]:
preds = model(inputs)
preds

tensor([[ 65.6279,  78.3223],
        [ 86.6758, 105.3553],
        [110.7386, 122.4156],
        [ 58.1081,  73.5472],
        [ 86.8367, 104.9583],
        [ 65.3130,  78.1868],
        [ 86.3116, 105.4180],
        [111.2050, 123.2813],
        [ 58.4230,  73.6827],
        [ 86.7874, 105.1565],
        [ 65.2638,  78.3850],
        [ 86.3609, 105.2198],
        [111.1028, 122.3529],
        [ 58.1574,  73.3490],
        [ 87.1516, 105.0939]], grad_fn=<AddmmBackward0>)

##**Loss function or cost function**
In `PyTorch`, we can use the built-in loss function `mse_loss`

In [42]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [43]:
# Define loss function
loss_fn = F.mse_loss

Now compute loss for the current model

In [44]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(349.5857, grad_fn=<MseLossBackward0>)


##**Optimizer (gradient descent)**
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "**stochastic gradient descent**". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

+ `model.parameters()` (weights and biases) is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step
+ we can specify a learning rate (`lr`) that controls the amount by which the parameters (weights and biases) are modified



In [45]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## **Train the model**

We are now ready to train the model. We'll follow the same process to implement gradient descent as we have done before:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The **only change** is that we'll work **batches of data** instead of processing the **entire training data** in every iteration. To avoid repetition, let's define a **utility function** `fit` that trains the model for a given number of epochs.

+ number of epochs `num_epochs`
+ model `model`
+ loss function `loss_fn`
+ optimizer `opt`
+ batch data from `dataloader` `train_dl`
+ input batch `xb`
+ output batch `yb`

Some things to note above:

* We use the data loader defined earlier to get **batches of data** for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero. 
This is the most important step! Once you pass the `PyTorch` model into the utility function `fit`, `opt.step()` will update parameters (weights and biases) in every epoch. After training is done, you will get an **updated model** with these updated parameters.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.


In [46]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients!!!!! (very important)
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Let's train the model for 100 epochs.

In [47]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 291.9106
Epoch [20/100], Loss: 191.4874
Epoch [30/100], Loss: 116.4991
Epoch [40/100], Loss: 67.6291
Epoch [50/100], Loss: 78.5976
Epoch [60/100], Loss: 42.4591
Epoch [70/100], Loss: 44.6003
Epoch [80/100], Loss: 40.4617
Epoch [90/100], Loss: 4.1298
Epoch [100/100], Loss: 11.8584


In [52]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.0322,  71.3844],
        [ 80.3286,  99.5371],
        [120.8689, 133.8515],
        [ 26.9509,  43.3918],
        [ 95.1725, 113.3712],
        [ 56.8793,  70.4134],
        [ 79.7856,  99.3730],
        [120.9881, 134.3523],
        [ 28.1037,  44.3628],
        [ 95.7822, 114.1781],
        [ 57.4891,  71.2203],
        [ 79.1758,  98.5661],
        [121.4120, 134.0156],
        [ 26.3411,  42.5848],
        [ 96.3253, 114.3422]], grad_fn=<AddmmBackward0>)

In [51]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have trained a reasonably good model to predict crop yields for apples and oranges by looking at the three features: **average temperature, rainfall, and humidity** in a region. We can use it to make predictions of crop yields for **new regions** by passing a batch containing a single row of input.

+ each row represents a training example

In [53]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.2759, 68.4604]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.